In [ ]:
from pathlib import Path

data_folder = Path('data')
if not (data_folder / 'Final Project data').exists():
    # Download and extract files
    from urllib.request import urlretrieve
    from zipfile import ZipFile

    data_folder.mkdir(exist_ok=True)

    zip_path = data_folder / 'Final Project data.zip'
    if not zip_path.exists() or zip_path.stat().st_size != 4874175804:
        def make_progress_hook():
            """Returns function hook to print download progress"""
            from time import perf_counter
            from sys import stdout

            def _progress_hook(count, block_size, total_size):
                nonlocal last_update, local_size, ema_speed, ema_size

                current_time = perf_counter()
                local_size += block_size
                time_interval = current_time - last_update
                if local_size >= total_size or time_interval > update_interval:
                    speed = (local_size - ema_size) / time_interval
                    ema_speed = ema_alpha * speed + (1 - ema_alpha) * ema_speed if local_size > 3e7 else speed
                    mins, secs = divmod(min(int((total_size - local_size) / ema_speed), 5999), 60)
                    last_update, ema_size = current_time, local_size
                    stdout.write(f"\rDownloading: {int(100 * local_size / total_size): 3}% [ETA: {mins:02}:{secs:02}] ")
                    stdout.flush()

            ema_alpha = 0.005
            update_interval = 0.2
            last_update = perf_counter()
            local_size = ema_speed = ema_size = 0
            return _progress_hook


        zip_url = 'https://remmar.s3.eu-central-1.amazonaws.com/Final+Project+data.zip'
        urlretrieve(zip_url, zip_path, reporthook=make_progress_hook())
        print('')

    print('Extracting...')
    with ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_folder)

In [28]:
from data import load_file
data, label = load_file(file_name_path= "data/Final Project data/Intra/train/task_motor_105923_1.h5")
print(label)
print(data.shape)

1
(248, 35624)


In [29]:
from data import preprocess_meg_sample
file_name_path = "data/Final Project data/Cross/train/task_working_memory_164636_7.h5"
data, label = load_file(file_name_path)
preprocess_meg_sample(data)

array([[ 0.35848824,  1.09445492,  2.10665066, ...,  0.73223247,
         0.68130328,  0.51004145],
       [-1.65286332, -1.90188809, -2.03713003, ..., -1.57736323,
        -1.54807689, -1.4020068 ],
       [ 0.75907758,  0.79060384,  0.82546458, ...,  0.84476225,
         0.66525369,  0.46801558],
       ...,
       [ 1.47625062,  1.74331856,  2.24407067, ...,  0.65807016,
         0.66537606,  0.72636419],
       [-1.26970342, -1.3127728 , -1.35584218, ..., -1.28935972,
        -1.34538767, -1.40348433],
       [ 0.96017504,  0.92555824,  0.9406922 , ...,  0.66420581,
         0.83909806,  0.72412342]])

In [30]:
from data import get_data_loaders
intra_loader_train, intra_loader_test = get_data_loaders(type="intra-subject", batch_size=2)
cross_loader_train, cross_loader_test = get_data_loaders(type="cross-subject", batch_size=2)

In [37]:
from models import RNN
from train import train_and_validate

from torch.nn import CrossEntropyLoss
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ExponentialLR
from torch import device, cuda
dev = device('cuda:0' if cuda.is_available() else 'cpu')

LEARNING_RATE= 0.1

rnn = RNN(input_size=248, hidden_size=248, num_layers=1, output_size=4).to(dev)
cross_entropy = CrossEntropyLoss()
optimizer = Adam(rnn.parameters(), lr=LEARNING_RATE)
optimizer = SGD(rnn.parameters(), lr=LEARNING_RATE)
scheduler = ExponentialLR(optimizer, gamma=0.9)

train_and_validate(criterion=cross_entropy, optimizer=optimizer, model=rnn, scheduler=scheduler, 
                   dataloader_train=intra_loader_train, dataloader_test=intra_loader_test, epochs=10)

[1,     1] loss: 0.011
[1,     6] loss: 0.054
[1,    11] loss: 0.061
[1,    16] loss: 0.045
Train Acc (epoch=0): 23/32	0.719
Val Acc: 2/8	0.250
[2,     1] loss: 0.269
[2,     6] loss: 0.015
[2,    11] loss: nan
[2,    16] loss: nan
Train Acc (epoch=1): 12/32	0.375
Val Acc: 2/8	0.250
[3,     1] loss: nan
[3,     6] loss: nan
[3,    11] loss: nan
[3,    16] loss: nan
Train Acc (epoch=2): 8/32	0.250
Val Acc: 2/8	0.250
[4,     1] loss: nan
[4,     6] loss: nan
[4,    11] loss: nan
[4,    16] loss: nan
Train Acc (epoch=3): 8/32	0.250
Val Acc: 2/8	0.250
[5,     1] loss: nan
[5,     6] loss: nan
[5,    11] loss: nan
[5,    16] loss: nan
Train Acc (epoch=4): 8/32	0.250
Val Acc: 2/8	0.250
[6,     1] loss: nan
[6,     6] loss: nan
[6,    11] loss: nan
[6,    16] loss: nan
Train Acc (epoch=5): 8/32	0.250
Val Acc: 2/8	0.250
[7,     1] loss: nan
[7,     6] loss: nan
[7,    11] loss: nan
[7,    16] loss: nan
Train Acc (epoch=6): 8/32	0.250
Val Acc: 2/8	0.250
[8,     1] loss: nan
[8,     6] loss: nan

In [ ]:
from models import BaselineCNN

from torch.nn import CrossEntropyLoss
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR

sample = next(iter(intra_loader_train))[0]
model = BaselineCNN(sample).to(dev)
criterion = CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = ExponentialLR(optimizer, gamma=0.9)